In [ ]:

# Combined U-Net + EfficientViT 
import os, math, time, random
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision, torchvision.transforms as T

# ---------------- Utilities ----------------
def count_params(m):
    total = sum(p.numel() for p in m.parameters())
    trainable = sum(p.numel() for p in m.parameters() if p.requires_grad)
    return total, trainable

def try_flops(model, input_size=(1,3,32,32)):
    try:
        from thop import profile
        device = next(model.parameters()).device
        dummy = torch.randn(*input_size).to(device)
        macs, _ = profile(model, inputs=(dummy,), verbose=False)
        return 2 * macs
    except Exception:
        return None

# ---------------- Droppath ----------------
class DropPath(nn.Module):
    def __init__(self, p=0.):
        super().__init__(); self.p = p
    def forward(self,x):
        if not self.training or self.p==0.: return x
        keep = 1 - self.p
        shape = (x.shape[0],) + (1,) * (x.ndim - 1)
        rand = x.new_empty(shape).bernoulli_(keep)
        return x / keep * rand

# ---------------- Core building blocks ----------------
class DWConv(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x, H, W):
        B, N, C = x.shape
        x2 = x.transpose(1,2).reshape(B, C, H, W)
        x2 = F.conv2d(x2, x2.new_ones((C,1,3,3))/9, groups=C, padding=1)
        return x2.reshape(B, C, -1).transpose(1,2)

class FFN(nn.Module):
    def __init__(self, dim, hidden_dim, drop=0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(drop),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(drop),
        )
    def forward(self, x): return self.net(x)

class CascadedGroupAttention(nn.Module):
    def __init__(self, dim, num_heads=4, qk_ratio=0.25):
        super().__init__()
        assert dim % num_heads == 0, "dim must be divisible by num_heads"
        self.h = num_heads
        self.head_dim = dim // num_heads
        self.qk_dim = max(1, int(self.head_dim * qk_ratio))
        self.q_projs = nn.ModuleList([nn.Linear(self.head_dim, self.qk_dim, bias=False) for _ in range(self.h)])
        self.k_projs = nn.ModuleList([nn.Linear(self.head_dim, self.qk_dim, bias=False) for _ in range(self.h)])
        self.v_projs = nn.ModuleList([nn.Linear(self.head_dim, self.head_dim, bias=False) for _ in range(self.h)])
        self.out = nn.Linear(dim, dim)

    def forward(self, x, H, W):
        # x: [B, N, C]
        B, N, C = x.shape
        xs = x.reshape(B, N, self.h, self.head_dim).permute(2,0,1,3)  # [h, B, N, head_dim]
        head_outs = []
        prev = None
        for j in range(self.h):
            xj = xs[j]  # [B, N, head_dim]
            if prev is not None:
                xj = xj + prev
            q = self.q_projs[j](xj)   # [B, N, qk_dim]
            k = self.k_projs[j](xj)
            v = self.v_projs[j](xj)   # [B, N, head_dim]
            sim = (q @ k.transpose(-2, -1)) / math.sqrt(max(1.0, float(self.qk_dim)))
            att = torch.softmax(sim, dim=-1)
            out_j = att @ v  # [B, N, head_dim]
            head_outs.append(out_j)
            prev = out_j.detach()
        out = torch.cat(head_outs, dim=-1)  # [B, N, C]
        return self.out(out)

class EfficientBlock(nn.Module):
    """A lightweight 'sandwich' block: FFN -> DW -> CGA -> FFN (pre-norm)"""
    def __init__(self, dim, num_heads=4, mlp_ratio=2.0, drop_path=0.0, qk_ratio=0.25):
        super().__init__()
        hidden = int(dim * mlp_ratio)
        self.norm1 = nn.LayerNorm(dim)
        self.ffn1 = FFN(dim, hidden)
        self.norm2 = nn.LayerNorm(dim)
        self.dw = DWConv()
        self.norm3 = nn.LayerNorm(dim)
        self.attn = CascadedGroupAttention(dim, num_heads=num_heads, qk_ratio=qk_ratio)
        self.norm4 = nn.LayerNorm(dim)
        self.ffn2 = FFN(dim, hidden)
        self.dp = DropPath(drop_path)

    def forward(self, x, H, W):
        x = x + self.dp(self.ffn1(self.norm1(x)))
        x = x + self.dp(self.dw(self.norm2(x), H, W))
        x = x + self.dp(self.attn(self.norm3(x), H, W))
        x = x + self.dp(self.ffn2(self.norm4(x)))
        return x

# ---------------- U-Net + EfficientViT Model ----------------
class UNetEfficientViT(nn.Module):
    def __init__(self,
                 in_ch=3,
                 num_classes=10,
                 enc_channels=(48, 96, 160),   
                 dec_channels=(96, 64),       
                 enc_depths=(1,2,3),
                 decoder_depths=(1,1),
                 heads=(2,4,4),
                 mlp_ratio=2.0,
                 qk_ratio=0.25,
                 drop_path_rate=0.1):
        super().__init__()

        self.stem = nn.Sequential(
            nn.Conv2d(in_ch, enc_channels[0], kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(enc_channels[0]),
            nn.ReLU(inplace=True),
            nn.Conv2d(enc_channels[0], enc_channels[0], kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(enc_channels[0]),
            nn.ReLU(inplace=True),
        )
        # encoder stages
        self.enc_channels = enc_channels
        self.enc_depths = enc_depths
        self.enc_stages = nn.ModuleList()
        self.enc_proj = nn.ModuleList()
        total_blocks = sum(enc_depths) + sum(decoder_depths)
        dp_rates = torch.linspace(0, drop_path_rate, total_blocks).tolist()
        dp_idx = 0
        for i, (C, L, h) in enumerate(zip(enc_channels, enc_depths, heads)):

            self.enc_proj.append(nn.Identity())
            blocks = nn.ModuleList()
            for _ in range(L):
                blocks.append(EfficientBlock(dim=C, num_heads=h, mlp_ratio=mlp_ratio, drop_path=dp_rates[dp_idx], qk_ratio=qk_ratio))
                dp_idx += 1
            self.enc_stages.append(blocks)

            if i < len(enc_channels) - 1:
                setattr(self, f"down{i}", nn.Sequential(
                    nn.Conv2d(C, enc_channels[i+1], kernel_size=3, stride=2, padding=1, bias=False),
                    nn.BatchNorm2d(enc_channels[i+1]),
                    nn.ReLU(inplace=True)
                ))

        self.up_convs = nn.ModuleList()
        self.dec_stages = nn.ModuleList()
        in_ch_dec = enc_channels[-1]
        for i, (C_out, L) in enumerate(zip(dec_channels, decoder_depths)):

            self.up_convs.append(nn.ConvTranspose2d(in_ch_dec, C_out, kernel_size=2, stride=2))
            blocks = nn.ModuleList()
            for _ in range(L):
                blocks.append(EfficientBlock(dim=C_out, num_heads=max(1, heads[-1]//2), mlp_ratio=mlp_ratio, drop_path=dp_rates[dp_idx], qk_ratio=qk_ratio))
                dp_idx += 1
            self.dec_stages.append(blocks)
            in_ch_dec = C_out


        self.refine = nn.Sequential(
            nn.Conv2d(in_ch_dec, in_ch_dec, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(in_ch_dec),
            nn.ReLU(inplace=True)
        )
        # classifier
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.head = nn.Linear(in_ch_dec, num_classes)

    def forward(self, x):
        B = x.shape[0]
        skips = []
        x = self.stem(x)  # [B, C0, H/2, W/2], H=W=32 -> now 16
        for i, blocks in enumerate(self.enc_stages):
            C = x.shape[1]; H = x.shape[2]; W = x.shape[3]
            # tokens
            tokens = x.reshape(B, C, -1).permute(0,2,1)  # [B, N, C]
            for blk in blocks:
                tokens = blk(tokens, H, W)
            x = tokens.permute(0,2,1).reshape(B, C, H, W)
            skips.append(x)   #
            if i < len(self.enc_stages) - 1:
                x = getattr(self, f"down{i}")(x)

        for i, (upconv, blocks) in enumerate(zip(self.up_convs, self.dec_stages)):
            x = upconv(x)  
            if skip.shape[1] != x.shape[1]:
                skip = F.interpolate(skip, size=(x.shape[2], x.shape[3]), mode='bilinear', align_corners=False)
                conv1x1 = nn.Conv2d(skip.shape[1], x.shape[1], kernel_size=1).to(x.device)
                skip = conv1x1(skip)
            x = torch.cat([x, skip], dim=1)  # [B, Cx + Cskip, H, W]
            conv_reduce = nn.Conv2d(x.shape[1], x.shape[1]//2 if (x.shape[1]//2)>0 else 1, kernel_size=1).to(x.device)
            x = F.relu(nn.BatchNorm2d(x.shape[1]//2 if (x.shape[1]//2)>0 else 1).to(x.device)(conv_reduce(x)))
            Bn, Cn, Hn, Wn = x.shape
            tokens = x.reshape(Bn, Cn, -1).permute(0,2,1)
            for blk in blocks:
                tokens = blk(tokens, Hn, Wn)
            x = tokens.permute(0,2,1).reshape(Bn, Cn, Hn, Wn)

        x = self.refine(x)
        x = self.global_pool(x).reshape(B, -1)
        out = self.head(x)
        return out

# ----------------  (CIFAR-10) ----------------
def get_cifar10_loaders(bs=128, num_workers=4):
    train_transforms = T.Compose([
        T.RandAugment(num_ops=2, magnitude=9),
        T.RandomCrop(32, padding=4),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize((0.4914,0.4822,0.4465),(0.2470,0.2435,0.2616)),
    ])
    test_transforms = T.Compose([
        T.ToTensor(),
        T.Normalize((0.4914,0.4822,0.4465),(0.2470,0.2435,0.2616)),
    ])
    train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transforms)
    test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transforms)
    return DataLoader(train, batch_size=bs, shuffle=True, num_workers=num_workers), \
           DataLoader(test, batch_size=bs, shuffle=False, num_workers=num_workers)

# ---------------- Mixup/CutMix  ----------------
def cutmix_box(size, lam):
    W, H = size[3], size[2]
    cut_rat = math.sqrt(1 - lam)
    cut_w, cut_h = int(W * cut_rat), int(H * cut_rat)
    cx, cy = np.random.randint(W), np.random.randint(H)
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2

def mixup_cutmix(x, y, alpha=1.0, cutmix_prob=0.5):
    lam = np.random.beta(alpha, alpha)
    rand_index = torch.randperm(x.size(0)).to(x.device)
    if np.random.rand() < cutmix_prob:
        bbx1, bby1, bbx2, bby2 = cutmix_box(x.size(), lam)
        x[:, :, bby1:bby2, bbx1:bbx2] = x[rand_index, :, bby1:bby2, bbx1:bbx2]
        lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (x.size(-1) * x.size(-2)))
    y_a, y_b = y, y[rand_index]
    return x, y_a, y_b, lam

# ---------------- Training  ----------------
def train_one_epoch(model, dl, optimizer, device, loss_fn, mixup_alpha=0.8):
    model.train()
    total_loss = 0.0
    top1 = 0
    top5 = 0
    n = 0
    pbar = tqdm(dl, leave=False)
    for xb, yb in pbar:
        xb, yb = xb.to(device), yb.to(device)
        xb, ya, yb2, lam = mixup_cutmix(xb, yb, alpha=mixup_alpha)
        optimizer.zero_grad()
        out = model(xb)
        loss = lam * loss_fn(out, ya) + (1 - lam) * loss_fn(out, yb2)
        loss.backward()
        optimizer.step()
        bs = out.size(0)
        total_loss += loss.item() * bs
        n += bs
        preds = out.topk(5, dim=1)[1]
        top1 += (lam * (preds[:, 0] == ya).sum() + (1 - lam) * (preds[:, 0] == yb2).sum()).item()

        for i in range(preds.size(0)):
            lab_a = ya[i].item(); lab_b = yb2[i].item()
            top5 += (lam * (lab_a in preds[i].tolist()) + (1 - lam) * (lab_b in preds[i].tolist()))
    return total_loss / n, top1 / n, top5 / n

def evaluate(model, dl, device, loss_fn):
    model.eval()
    total_loss = 0.0
    top1 = 0
    top5 = 0
    n = 0
    with torch.no_grad():
        for xb, yb in dl:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            loss = loss_fn(out, yb)
            bs = out.size(0)
            total_loss += loss.item() * bs
            n += bs
            preds = out.topk(5, dim=1)[1]
            top1 += (preds[:, 0] == yb).sum().item()
            for i in range(preds.size(0)):
                if yb[i].item() in preds[i].tolist():
                    top5 += 1
    return total_loss / n, top1 / n, top5 / n

# ---------------- Plot  ----------------
def plot_history(hist, out_dir='outputs'):
    os.makedirs(out_dir, exist_ok=True)
    epochs = len(hist['train_loss'])
    x = list(range(1, epochs + 1))
    plt.figure(figsize=(6,4))
    plt.plot(x, hist['train_loss'], label='train_loss')
    plt.plot(x, hist['val_loss'], label='val_loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.grid(True); plt.title('Loss')
    plt.savefig(os.path.join(out_dir, 'loss.png'), dpi=200); plt.close()

    plt.figure(figsize=(6,4))
    plt.plot(x, np.array(hist['train_top1']) * 100, label='train_top1')
    plt.plot(x, np.array(hist['val_top1']) * 100, label='val_top1')
    plt.plot(x, np.array(hist['val_top5']) * 100, label='val_top5')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy (%)'); plt.legend(); plt.grid(True); plt.title('Accuracy')
    plt.savefig(os.path.join(out_dir, 'accuracy.png'), dpi=200); plt.close()


def main():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    batch_size = 128
    epochs = 150
    lr = 3e-4
    wd = 0.05
    patience = 15

    model = UNetEfficientViT(
        in_ch=3,
        num_classes=10,
        enc_channels=(48, 96, 160),   
        dec_channels=(160, 96),       
        enc_depths=(1,2,2),
        decoder_depths=(1,1),
        heads=(2,4,4),
        mlp_ratio=2.0,
        qk_ratio=0.25,
        drop_path_rate=0.1
    ).to(device)

    total_params, trainable = count_params(model)
    print(f"Params: {total_params:,} ({total_params/1e6:.3f}M) | Trainable: {trainable:,}")
    flops = try_flops(model, input_size=(1,3,32,32))
    if flops is not None:
        print(f"FLOPs (approx): {flops/1e6:.1f}M")

    train_loader, test_loader = get_cifar10_loaders(bs=batch_size, num_workers=4)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1)

    history = {'train_loss':[], 'train_top1':[], 'train_top5':[], 'val_loss':[], 'val_top1':[], 'val_top5':[]}

    best_val = 0.0
    wait = 0
    os.makedirs('outputs', exist_ok=True)

    for ep in range(1, epochs+1):
        t0 = time.time()
        train_loss, train_t1, train_t5 = train_one_epoch(model, train_loader, optimizer, device, loss_fn, mixup_alpha=0.8)
        val_loss, val_t1, val_t5 = evaluate(model, test_loader, device, loss_fn)
        scheduler.step()

        history['train_loss'].append(train_loss); history['train_top1'].append(train_t1); history['train_top5'].append(train_t5)
        history['val_loss'].append(val_loss); history['val_top1'].append(val_t1); history['val_top5'].append(val_t5)

        improved = val_t1 > best_val
        if improved:
            best_val = val_t1
            wait = 0
            torch.save(model.state_dict(), os.path.join('outputs', 'best_unet_efficientvit_cifar10.pth'))
        else:
            wait += 1

        t1 = time.time()
        print(f"E{ep:03d}/{epochs} | TL {train_loss:.3f} | VL {val_loss:.3f} | T1 {train_t1*100:.2f}% | V1 {val_t1*100:.2f}% | V5 {val_t5*100:.2f}% | Wait {wait} | time {t1-t0:.1f}s")

        if wait >= patience:
            print(f"Early stopping at epoch {ep} (no val acc improvement for {patience} epochs).")
            break

    torch.save(model.state_dict(), os.path.join('outputs', 'last_unet_efficientvit_cifar10.pth'))
    plot_history(history, out_dir='outputs')
    print(f"Done. Best val Top-1: {best_val*100:.2f}%")

if __name__ == '__main__':
    main()


Params: 1,477,610 (1.478M) | Trainable: 1,477,610


100%|██████████| 170M/170M [00:02<00:00, 62.4MB/s]


E001/150 | TL 2.307 | VL 2.253 | T1 10.87% | V1 14.42% | V5 59.71% | Wait 0 | time 45.7s


E002/150 | TL 2.271 | VL 2.158 | T1 13.57% | V1 18.96% | V5 71.73% | Wait 0 | time 46.5s


E003/150 | TL 2.230 | VL 2.108 | T1 15.52% | V1 19.10% | V5 77.75% | Wait 0 | time 48.1s


...

E138/150 | TL 1.894 | VL 1.291 | T1 44.11% | V1 74.27% | V5 97.96% | Wait 11 | time 48.0s


E139/150 | TL 1.839 | VL 1.274 | T1 46.53% | V1 75.03% | V5 97.98% | Wait 12 | time 48.0s


E140/150 | TL 1.876 | VL 1.283 | T1 44.86% | V1 74.37% | V5 98.12% | Wait 13 | time 48.1s


E141/150 | TL 1.869 | VL 1.275 | T1 45.36% | V1 74.64% | V5 98.08% | Wait 14 | time 48.3s


E142/150 | TL 1.878 | VL 1.276 | T1 44.96% | V1 74.36% | V5 98.19% | Wait 15 | time 48.0s
Early stopping at epoch 142 (no val acc improvement for 15 epochs).
Done. Best val Top-1: 75.22%
